<a href="https://colab.research.google.com/github/LucasCarmoPaschoal/Convolu-o-FFT/blob/main/convolu_o_fft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile fft_example.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define PI 3.14159265358979323846
// estruta de um numero complexo
typedef struct {
    double real;
    double imag;
} Complex;
// funçao multiplicação complex
Complex complex_mult(Complex a, Complex b) {
    Complex res;
    res.real = a.real * b.real - a.imag * b.imag;
    res.imag = a.real * b.imag + a.imag * b.real;
    return res;
}
// funçao soma complex
Complex complex_add(Complex a, Complex b) {
    Complex res;
    res.real = a.real + b.real;
    res.imag = a.imag + b.imag;
    return res;
}
// funçao subitração complex
Complex complex_sub(Complex a, Complex b) {
    Complex res;
    res.real = a.real - b.real;
    res.imag = a.imag - b.imag;
    return res;
}
// função que encontra primeira potencia de 2 maior que n
int next_power_of_two(int n) {
    int power = 1;
    while (power < n) power *= 2;
    return power;
}
// função recursiva fft por decimação no tempo recebendo um sinal entrada  e o tamanho
void FFT(Complex *x, int N) {
    if (N <= 1) return;

    Complex *even = malloc(N/2 * sizeof(Complex));
    Complex *odd  = malloc(N/2 * sizeof(Complex));

    for (int i = 0; i < N/2; i++) {
        even[i] = x[2*i];
        odd[i]  = x[2*i + 1];
    }

    FFT(even, N/2);
    FFT(odd, N/2);

    for (int k = 0; k < N/2; k++) {
        double angle = -2 * PI * k / N;
        Complex twiddle = {cos(angle), sin(angle)};
        Complex t = complex_mult(twiddle, odd[k]);
        x[k]       = complex_add(even[k], t);
        x[k+N/2]   = complex_sub(even[k], t);
    }

    free(even);
    free(odd);
}

void IFFT(Complex *x, int N) {
    if (N <= 1) return;

    Complex *even = malloc(N/2 * sizeof(Complex));
    Complex *odd  = malloc(N/2 * sizeof(Complex));

    for (int i = 0; i < N/2; i++) {
        even[i] = x[2*i];
        odd[i]  = x[2*i + 1];
    }

    IFFT(even, N/2);
    IFFT(odd, N/2);

    for (int k = 0; k < N/2; k++) {
        double angle = 2 * PI * k / N;
        Complex twiddle = {cos(angle), sin(angle)};
        Complex t = complex_mult(twiddle, odd[k]);
        x[k]       = complex_add(even[k], t);
        x[k+N/2]   = complex_sub(even[k], t);
    }

    free(even);
    free(odd);
}

void normalize_ifft(Complex* x, int N) {
    for (int i = 0; i < N; i++) {
        x[i].real /= N;
        x[i].imag /= N;
    }
}

// Convolução via FFT
void convolution(Complex* x, int Nx, Complex* h, int Nh, Complex* y_out) {
    int N = Nx + Nh - 1;
    int Nfft = next_power_of_two(N);

    Complex *X = calloc(Nfft, sizeof(Complex));
    Complex *H = calloc(Nfft, sizeof(Complex));
    Complex *Y = calloc(Nfft, sizeof(Complex));

    // Copia com zero-padding
    for (int i = 0; i < Nx; i++) X[i] = x[i];
    for (int i = 0; i < Nh; i++) H[i] = h[i];

    FFT(X, Nfft);
    FFT(H, Nfft);

    for (int i = 0; i < Nfft; i++) {
        Y[i] = complex_mult(X[i], H[i]);
    }

    IFFT(Y, Nfft);
    normalize_ifft(Y, Nfft);
    for (int i = 0; i < N; i++) {
        y_out[i] = Y[i];
    }

    free(X); free(H); free(Y);
}

int main() {
    Complex x[4] = {{1,0}, {2,0}, {3,0}, {4,0}};
    Complex h[3] = {{1,0}, {1,0}, {1,0}};
    int Nx = 4, Nh = 3;
    int Nout = Nx + Nh - 1;

    Complex* y = calloc(Nout, sizeof(Complex));

    convolution(x, Nx, h, Nh, y);

    printf("Resultado da convolução linear:\n");
    for (int i = 0; i < Nout; i++) {
        printf("y[%d] = %.3f + %.3fi\n", i, y[i].real, y[i].imag);
    }

    free(y);
    return 0;
}


Overwriting fft_example.c


In [ ]:
!gcc fft_example.c -o fft_example -lm
!./fft_example

Resultado da convolução linear:
y[0] = 1.000 + -0.000i
y[1] = 3.000 + -0.000i
y[2] = 6.000 + -0.000i
y[3] = 9.000 + -0.000i
y[4] = 7.000 + 0.000i
y[5] = 4.000 + 0.000i
